In [1]:
# Intall packages we'll use and print success when finished. 
# Not all these packages will be used in this notebook, but it is good practice

import matplotlib as plt
import pandas as pd
import seaborn as sns
import numpy as np
print("Success!!")

Success!!


# Metrics

The amount of data that Facebook gives its users is immense and is broken down by Daily, Weekly, 28 Day period (Facebook monthly), and lifetime values. If you download all of a page's data you get each metric broken up by the previously mentioned time frames. As such, some pre-prep is ideal. My recommendation is to only pull daily data and lifetime data of metrics you care about i.e. Lifetime Likes by Gender and Age. 

To do this, go to the Facebook page you manage, click on insights, then on export data. Once there you'll see a new floating window that will ask you what data you want to export by Page, Post, or Video data. Ensure that "Page Data" is selected and then click on "make new custom layout." This will allow you to tell Facebook what data you are looking to download.

In this notebook we will be dealing with the following data, but the steps taken here will be the same for which ever data you choose.

- Daily New Likes
- Daily Unlikes
- Daily New Follows                                  
- Daily Page Engaged Users
- Daily Post Engagements
- Daily Total Reach
- Daily Nonviral Reach
- Daily Organic Reach
- Daily Viral Reach
- Daily Paid Reach
- Daily People Talking About This
- Daily Total: total action count per Page
- Daily Total website click count per Page
- Lifetime Total Likes 
- Lifetime Likes by Country
- Lifetime likes by Age and Gender
- Daily Reactions per page by type

Once you have the data you want to export chosen, you'll then need to pick a date range. Facebook limits how much data you can export/download at one time by time. As such, I recommend keeping it neat and tidy by downloading entire quarters or months at one time. Once downloaded, you're ready to import that data.


## Read CSVs into DFs

In [2]:
# read the CSV(s) into a dataframe(s)
Q12019 = pd.read_csv("C:/Users/anthony/Documents/Data_Science/Facebook Data/Q12019.csv")
Q22019 = pd.read_csv("C:/Users/anthony/Documents/Data_Science/Facebook Data/Q22019.csv")
Q32019 = pd.read_csv("C:/Users/anthony/Documents/Data_Science/Facebook Data/Q32019.csv")
Q42019 = pd.read_csv("C:/Users/anthony/Documents/Data_Science/Facebook Data/Q42019.csv")
Q12020 = pd.read_csv("C:/Users/anthony/Documents/Data_Science/Facebook Data/Q12020.csv")
Q22020 = pd.read_csv("C:/Users/anthony/Documents/Data_Science/Facebook Data/Q22020.csv")

# Use pandas concatenate function to combine the CSVs into a single dataframe like..
# a SQL union that doesn't require the all the columns to match 
CSP = pd.concat([Q12019, Q22019, Q32019, Q42019, Q12020, Q22020])

# Print quick look
CSP

,Date,Daily New Likes,Daily Unlikes,Daily New Follows,Daily Page Engaged Users,Daily Post Engagements,Daily Total Reach,Daily Nonviral Reach,Daily Organic Reach,Daily Viral Reach,...,Lifetime Likes by Gender and Age - F.55-64,Lifetime Likes by Gender and Age - F.65+,Lifetime Likes by Gender and Age - M.18-24,Lifetime Likes by Gender and Age - M.25-34,Lifetime Likes by Gender and Age - M.35-44,Lifetime Likes by Gender and Age - M.45-54,Lifetime Likes by Gender and Age - M.55-64,Lifetime Likes by Gender and Age - M.65+,Lifetime Likes by Gender and Age - U.35-44,Lifetime Likes by Country - ES
0,NaN,Daily: The number of new people who have liked...,Daily: The number of Unlikes of your Page (Uni...,Daily: The number of new people who have follo...,Daily: The number of people who engaged with y...,Daily: The number of times people have engaged...,Daily: The number of people who had any conten...,Daily: The number of people who had any conten...,Daily: The number of people who had any conten...,Daily: The number of people who had any conte...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2020-06-26,NaN,NaN,NaN,3,6,683,37,37,NaN,...,16.0,8.0,5.0,47.0,42.0,36.0,22.0,13.0,1.0,NaN
88,2020-06-27,3,NaN,4,68,221,2276,47,1796,1763,...,16.0,8.0,5.0,49.0,42.0,36.0,22.0,13.0,1.0,NaN
89,2020-06-28,NaN,NaN,NaN,16,24,945,12,443,434,...,16.0,8.0,5.0,49.0,42.0,36.0,22.0,13.0,1.0,NaN
90,2020-06-29,NaN,NaN,NaN,21,33,704,94,182,92,...,16.0,8.0,5.0,49.0,42.0,36.0,22.0,13.0,1.0,NaN


Lets start cleaning the data, First let's get rid of the description row, reset the index, and fill in the NaN values with 0 since we know this data represents likes, reactions, etc

In [3]:
CSP.drop(0, inplace = True)
CSP.reset_index(drop=True, inplace=True)
CSP.fillna(0, inplace=True)

Every value execpt the date column needs to be in number format. We can quickly apply this to the whole data frame by using the df.apply(pd.to_numeric) function and pass errors = ignore to ignore any value that isn't convertable to a numeric value. After, we can then covert the date coulmn to a date fomat for easier analysis.

In [4]:
CSP = CSP.apply(pd.to_numeric, errors='ignore')
CSP['Date'] = pd.to_datetime(CSP['Date'])
# Print the types to check
CSP.dtypes

Date                                              datetime64[ns]
Daily New Likes                                            int64
Daily Unlikes                                              int64
Daily New Follows                                          int64
Daily Page Engaged Users                                   int64
Daily Post Engagements                                     int64
Daily Total Reach                                          int64
Daily Nonviral Reach                                       int64
Daily Organic Reach                                        int64
Daily Viral Reach                                          int64
Daily Paid Reach                                           int64
Daily People Talking About This                            int64
Daily Total: total action count per Page                   int64
Daily Total website click count per Page                   int64
Lifetime Total Likes                                       int64
Daily: total post reactio

Now that we have the columns in the right format, let's rename all the columns to make it easier for us to read and deliver our data to people not familiar with it. For example we can rename Daily: total post reactions of a page. - wow to Daily Wows. We will do this by passing the df.columns function. Make sure you have your columns name in the order you want to see them.

In [5]:
# Get the column list
CSP.columns

Index(['Date', 'Daily New Likes', 'Daily Unlikes', 'Daily New Follows',
       'Daily Page Engaged Users', 'Daily Post Engagements',
       'Daily Total Reach', 'Daily Nonviral Reach', 'Daily Organic Reach',
       'Daily Viral Reach', 'Daily Paid Reach',
       'Daily People Talking About This',
       'Daily Total: total action count per Page',
       'Daily Total website click count per Page', 'Lifetime Total Likes',
       'Daily: total post reactions of a page. - anger',
       'Daily: total post reactions of a page. - haha',
       'Daily: total post reactions of a page. - like',
       'Daily: total post reactions of a page. - love',
       'Daily: total post reactions of a page. - sorry',
       'Daily: total post reactions of a page. - wow',
       'Lifetime Likes by Country - AU', 'Lifetime Likes by Country - BE',
       'Lifetime Likes by Country - CA', 'Lifetime Likes by Country - DE',
       'Lifetime Likes by Country - DO', 'Lifetime Likes by Country - DZ',
       'Lifeti

In [6]:
# Copy the column list and rename as needed.

CSP.columns = ['Date', 'Page Likes', 'Unlikes', 'New Follows',
       'Engaged Users', 'Post Engagements',
       'Total Reach', 'Nonviral Reach', 'Organic Reach',
       'Viral Reach', 'Paid Reach',
       'Talking About This',
       'Action Counts',
       'Website Clicks', 'Lifetime Total Likes',
       'Anger',
       'Haha',
       'Likes',
       'Love',
       'Sad',
       'Wow',
       'Lifetime Likes - AU', 'Lifetime Likes - BE',
       'Lifetime Likes - CA', 'Lifetime Likes - DE',
       'Lifetime Likes - DO', 'Lifetime Likes - DZ',
       'Lifetime Likes - ET', 'Lifetime Likes - GB',
       'Lifetime Likes - GH', 'Lifetime Likes - IN',
       'Lifetime Likes - MG', 'Lifetime Likes - MX',
       'Lifetime Likes - NZ', 'Lifetime Likes - PK',
       'Lifetime Likes - SN', 'Lifetime Likes - SO',
       'Lifetime Likes TH', 'Lifetime Likes - US',
       'Lifetime Likes - ZA',
       'Lifetime Likes - F.18-24',
       'Lifetime Likes - F.25-34',
       'Lifetime Likes - F.35-44',
       'Lifetime Likes - F.45-54',
       'Lifetime Likes - F.55-64',
       'Lifetime Likes - F.65+',
       'Lifetime Likes - M.18-24',
       'Lifetime Likes - M.25-34',
       'Lifetime Likes - M.35-44',
       'Lifetime Likes - M.45-54',
       'Lifetime Likes - M.55-64',
       'Lifetime Likes - M.65+',
       'Lifetime Likes - U.35-44',
       'Lifetime Likes - ES']

# Print the columns to see if the renamed columns took
CSP.columns

Index(['Date', 'Page Likes', 'Unlikes', 'New Follows', 'Engaged Users',
       'Post Engagements', 'Total Reach', 'Nonviral Reach', 'Organic Reach',
       'Viral Reach', 'Paid Reach', 'Talking About This', 'Action Counts',
       'Website Clicks', 'Lifetime Total Likes', 'Anger', 'Haha', 'Likes',
       'Love', 'Sad', 'Wow', 'Lifetime Likes - AU', 'Lifetime Likes - BE',
       'Lifetime Likes - CA', 'Lifetime Likes - DE', 'Lifetime Likes - DO',
       'Lifetime Likes - DZ', 'Lifetime Likes - ET', 'Lifetime Likes - GB',
       'Lifetime Likes - GH', 'Lifetime Likes - IN', 'Lifetime Likes - MG',
       'Lifetime Likes - MX', 'Lifetime Likes - NZ', 'Lifetime Likes - PK',
       'Lifetime Likes - SN', 'Lifetime Likes - SO', 'Lifetime Likes TH',
       'Lifetime Likes - US', 'Lifetime Likes - ZA',
       'Lifetime Likes - F.18-24', 'Lifetime Likes - F.25-34',
       'Lifetime Likes - F.35-44', 'Lifetime Likes - F.45-54',
       'Lifetime Likes - F.55-64', 'Lifetime Likes - F.65+',
       '

Now this notebook is ready for reuse to clean Facebook data quickly. As new months are completed, you can simply download the CSVs of the data and concanate them together. Be sure to download the combined CSV using the cell below after merging.

In [8]:
import tkinter as tk
from tkinter import filedialog

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
canvas1.pack()

def exportCSV ():
    global df

    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
#Replace CSP with your DF
    CSP.to_csv (export_file_path, index = False, header=True)

saveAsButton_CSV = tk.Button(text='Export CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButton_CSV)

root.mainloop()